# Setup object detection model

## 1. Create folder hierarchy

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pickle

### [OBJECT DETECTION MODEL ZOO](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [ ]:
# CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
# PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_ssd' 
# PRETRAINED_MODEL_NAME = 'ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_ssd_resnet152' 
# PRETRAINED_MODEL_NAME = 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz'

CUSTOM_MODEL_NAME = 'my_ssd_resnet50' 
PRETRAINED_MODEL_NAME = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_ssd_resnet50' 
# PRETRAINED_MODEL_NAME = 'ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_centernet_mobilenetv2fpn' 
# PRETRAINED_MODEL_NAME = 'centernet_mobilenetv2_fpn_kpts'
# PRETRAINED_MODEL_URL = 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1?tf-hub-format=compressed'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),

    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join(paths['MODEL_PATH'], CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': TF_RECORD_SCRIPT_NAME, 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'PRETRAINED_MODEL_CONFIG': os.path.join(paths['PRETRAINED_MODEL_PATH'],
                                            PRETRAINED_MODEL_NAME, 'pipeline.config'),
    'PRETRAINED_MODEL_CKP0': os.path.join('Tensorflow', 'workspace','pre-trained-models',
                                          PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
}

# create workspace directories
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
            
pickle.dump(files, open('files.pickle', 'wb'))
pickle.dump(paths, open('paths.pickle', 'wb'))

## 2. Download pretrained model

In [ ]:
!wget --no-proxy {PRETRAINED_MODEL_URL} -O {PRETRAINED_MODEL_NAME}.tar.gz
!mv {PRETRAINED_MODEL_NAME + '.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

## 3. Clone tensorflow and build models API

In [ ]:
!git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Build API buffers
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection
!cd Tensorflow/models/research && cp object_detection/packages/tf2/setup.py . && python3 -m pip install .

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python3 {VERIFICATION_SCRIPT}

In [ ]:
import object_detection